In [1]:
%load_ext autoreload
%config Completer.use_jedi = False

In [5]:
%autoreload 2
import FEMOL
import numpy as np
np.set_printoptions(linewidth=200)
np.set_printoptions(suppress=True)
from IPython.display import clear_output
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from scipy.interpolate import interp1d, griddata
import matplotlib.cm as colormaps
import matplotlib.colors
from mpl_toolkits.axes_grid1 import Divider, Size
import meshio
import time
import runpy

# Development Notebook for the FEMOL Project

In [600]:
# Mean stiffnesses
h_vec = mesh.point_data['zc'].copy()
h_vec[h_area > mesh.point_data['zc']] = h_vec.min()
h_bar = h_vec.mean()

# Optimized layup
plies = [90, 90]
carbon = FEMOL.materials.general_carbon()
layup = FEMOL.laminate.Layup(material=carbon, plies=plies, symetric=False, h_core=0, z_core = h_bar + carbon.hi)
D_mat_coat = layup.D_mat
A_mat_coat = layup.A_mat*0.5
flax = FEMOL.materials.general_flax()
layup2 = FEMOL.laminate.Layup(material=flax, plies=[0,0,0], symetric=True, h_core=0, z_core=0)
D_mat_base = layup2.D_mat
A_mat_base = layup2.A_mat

# reference layup
plies_ref = [90]
h_ref = 0.025
layup_ref1 = FEMOL.laminate.Layup(material=carbon, plies=plies_ref, symetric=False, h_core=0, z_core = h_ref + carbon.hi/2)
D_mat_ref_coat = layup_ref1.D_mat
A_mat_ref_coat = layup_ref1.A_mat




In [178]:
# Modal Analysis of the reference plate for the case study
# dimensions
a = 0.201
b = 0.201
# flax base
t_flax = 0.002
hi_flax = t_flax/n_plies
flax_plies = 6*[0]
flax = FEMOL.materials.general_flax()
flax.rho = 1130
flax.hi = hi_flax
flax_lam = FEMOL.laminate.Layup(material=flax, plies=flax_plies, symetric=False, z_core= -0.0036 + flax_lam.h/2)
# PLA core
infill=0.25
PLA = FEMOL.materials.printed_PLA(infill=infill)
t_PLA = 0.005
# Carbon top layer
t_carbon = 0.0002
hi_carbon = t_carbon
carbon_plies = [90]
carbon = FEMOL.materials.general_carbon()
carbon.hi = hi_carbon
carbon.rho = 1550
carbon_plies = [90]
carbon_lam = FEMOL.laminate.Layup(material=carbon, plies=carbon_plies, symetric=False, z_core= 0.0036 - carbon.hi/2)

# Analysis without consideration for PLA core
mesh = FEMOL.mesh.rectangle_Q4(a, b, 15, 15)
problem = FEMOL.FEM_Problem(mesh=mesh, physics='modal', model='plate')
problem.define_materials(flax, carbon)
problem.define_tensors(flax_lam, carbon_lam)
w, v = problem.solve(filtre=0)

ind = np.argmax([FEMOL.utils.MAC(ref_vector, vi) for vi in v])
print('No core eigenfrequency : ', w[ind])

solving using scipy
solved in :  1.2561848163604736  s
No core eigenfrequency :  499.9519787140608
